# Lab 07 - optymalizacja hiperparametrów modeli

Znaczna większość modeli posiada daotkowe zewnętrzne możliwości dostrajania za pomocą hiperparametrów. Proces poszukiwania kombinacji hiperparametrów dającej najbardziej zastysfakcjonujące wyniki nazywany jest **optymalizacją hiperparametrów**.


Najprostsze podejście do optymalizacji hiperparametrów polega na manualnym dopasowywaniu ich wartości, a nastepnie na porównywaniu wyników uzyskanych na podzbiorze testowym. Biblioteka Scikit-learn zawiera szereg klas umożliwiających automatyczną optymalizację hiperparametrów korzystając z różnych podejść.

## Metoda przeszukiwania siatki

Metoda przeszukiwania siatki to deterministyczne podejście do optymalizacji hiperparametrów polegające na testowaniu wszystkich możliwych ich kombinacji. Do realizacji procesu przeszukiwania siatki w bibliotece Scikit-learn dostępna jest klasa [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Warto zauważyć, że wszystkie klasy estymatorów posiadają metodę *fit*, co oznacza że mogą być traktowane jak tradycyjne estymatory niewymagające manualnego wskazywania hiperparametrów. Co więcej, klasy te przeprowadzają domyślnie test krzyżowy.

In [9]:
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import Normalizer

In [10]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='pandas')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

Zbiór danych **MNIST_784** to jedno z najbardziej znanych i szeroko używanych zbiorów danych w dziedzinie uczenia maszynowego, szczególnie w zadaniach klasyfikacji obrazów. Zawiera obrazy ręcznie napisanych cyfr od 0 do 9.

### Struktura zbioru danych MNIST:
- **Liczba próbek**: Zbiór składa się z **70,000 próbek** zazwyczaj podzielonych na:
  - **60,000 próbek** treningowych (obrazów).
  - **10,000 próbek** testowych (obrazów).
  
- **Wymiary danych**:
  - Każdy obraz w zbiorze MNIST ma rozdzielczość **28 x 28 pikseli**.
  - Zatem każdy obraz to wektor o długości **784** (28 x 28 = 784), ponieważ każdy piksel obrazu jest reprezentowany przez jedną wartość (skala szarości, od 0 do 255).
  
- **Etykiety**:
  - Każdy obraz przedstawia jedną z dziesięciu cyfr od 0 do 9.
  - Wartości etykiet to liczby całkowite z zakresu [0, 9], które wskazują, jaka cyfra jest reprezentowana na obrazie.

### Opis danych:
- **Obrazy**: Każdy obraz to macierz o wymiarach 28x28, gdzie każda komórka zawiera wartość od 0 (czarny) do 255 (biały). Obrazy są w skali szarości, co oznacza, że każdy piksel ma jedną wartość reprezentującą jasność.
- **Etykiety**: Etykiety to liczby całkowite z zakresu 0-9, które wskazują, jaką cyfrę przedstawia dany obraz.

### Przykłady zastosowań:
- **Klasyfikacja obrazów**: Celem najczęściej jest zbudowanie modelu klasyfikacyjnego, który przypisuje etykietę cyfry (0-9) do każdego obrazu.
- **Rozpoznawanie pisma ręcznego**: MNIST jest powszechnie używany jako standardowy zbiór do testowania algorytmów rozpoznawania pisma ręcznego.

### Właściwości:
- **Wysoka jakość danych**: Obrazy są wystarczająco małe, aby mogły być łatwo przetwarzane na standardowych komputerach, ale jednocześnie oferują wystarczającą złożoność, by stanowić wyzwanie dla algorytmów uczenia maszynowego.
- **Prosty, ale skuteczny zbiór testowy**: Zbiór jest dobrze znany i szeroko stosowany w literaturze, co sprawia, że jest doskonałym punktem wyjścia do testowania nowych algorytmów.

In [11]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [12]:
preprocessing_pipeline = make_pipeline(
    Normalizer(),
)

* Tworzymy potok (`pipeline`) do wstępnego przetwarzania danych. W tym przypadku jest to tylko jeden krok: normalizacja za pomocą `Normalizer()`.
* `Normalizer()` normalizuje dane, tzn. skaluje je tak, by ich normy (według normy L1, L2 lub max) były równe 1. Przydatne, gdy dane mają różne skale, a algorytmy klasyfikacji mogą być wrażliwe na różnice w skali cech.

Normy L1, L2 i max to różne metody skalowania danych w procesie normalizacji, które różnią się sposobem obliczania i interpretowania długości wektora (czyli odległości) w przestrzeni wielowymiarowej. Oto ich szczegóły:

### 1. **Norma L1 (Manhattan)**:
Norma L1, znana również jako norma Manhattan lub taksówkowa, oblicza sumę wartości bezwzględnych składników wektora:
$
\|x\|_1 = \sum_{i=1}^n |x_i|
$
- **Przykład**: Jeśli wektor to $([1, -2, 3])$, to norma L1 będzie wynosić $( |1| + |-2| + |3| = 6 )$.
- **Zastosowanie**: Norma L1 jest często używana w zadaniach związanych z minimalizowaniem funkcji kosztu w problemach związanych z wyborem cech lub w regularizacji (np. Lasso), ponieważ skłania do zerowania niektórych współczynników.

### 2. **Norma L2 (Euklidesowa)**:
Norma L2, znana również jako norma Euklidesowa, oblicza pierwiastek kwadratowy z sumy kwadratów składników wektora:
$
\|x\|_2 = \sqrt{\sum_{i=1}^n x_i^2}
$
- **Przykład**: Dla wektora $([1, -2, 3])$, norma L2 wynosi $( \sqrt{1^2 + (-2)^2 + 3^2} = \sqrt{1 + 4 + 9} = \sqrt{14} \approx 3.74 )$.
- **Zastosowanie**: Norma L2 jest powszechnie stosowana w algorytmach takich jak regresja liniowa (Ridge) i w zadaniach optymalizacyjnych, gdzie chcemy zminimalizować wielkość wektora, nie eliminując całkowicie żadnej cechy.

### 3. **Norma Max (Norma maksimum)**:
Norma max (znana również jako norma supremum) oblicza maksymalną wartość bezwzględną składników wektora:
$
\|x\|_{\infty} = \max_i |x_i|
$
- **Przykład**: Dla wektora $([1, -2, 3])$, norma max wynosi $( \max(|1|, |-2|, |3|) = 3 )$.
- **Zastosowanie**: Norma max jest używana, gdy chcemy, by wartość cechy o największej wartości absolutnej miała dominujący wpływ na normalizację. Jest to także popularna norma w zadaniach, w których istotne jest utrzymanie ekstremalnych wartości cech.

In [13]:
full_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('classification', RandomForestClassifier(random_state=42)),
])

* `full_pipeline` to pełny potok, który zawiera dwa etapy:
* **Preprocessing**: aplikacja wcześniej stworzonego potoku normalizacji.
* **Classification**: klasyfikator `RandomForestClassifier`, który jest używany do trenowania modelu lasu losowego.
* Parametr `random_state=42` w `RandomForestClassifier` zapewnia reprodukowalność wyników.

Metoda przeszukiwania siatki - podobnie jak inne metody optymalizacji hiperparametrów - polega na jawnie wskazanych hiperparametrach oraz zbiorach lub zakresach ich wartości. Dla potrzeb biblioteki *Scikit-learn* należy je wskazać jako kolekcję zawierającą słowniki z przypisanymi wartościami kombinacji do przetestowania. Każdy z hiperparametrów musi mieć również poprzedzoną nazwę za pomocą prefixu będącego nazwą etapu w potoku końcowym.

In [14]:
hiperparam_grid = [{
      'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
      'classification__n_estimators': (16, 32, 64),
      'classification__max_features': range(4, 14, 5),
  }, {
      'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
      'classification__n_estimators': (64, 128, 256),
      'classification__max_features': ('sqrt', 'log2'),
}]

`hiperparam_grid` to lista słowników z różnymi zestawami hiperparametrów, które będą testowane przez `GridSearchCV`.
Każdy słownik zawiera:
* `preprocessing__normalizer__norm`: różne możliwe normy do zastosowania w Normalizer (L1, L2, max).
* `classification__n_estimators`: różna liczba drzew w lesie losowym (16, 32, 64, 128, 256).
* `classification__max_features`: różne sposoby wyboru cech w klasyfikatorze (np. liczba cech lub funkcje takie jak 'sqrt' i 'log2').

Warto zawrócić uwagę na prefix każdego hiperparametru odzielony podwójny `_`, który odpowiednio wskazuje do którego kroku należy dany hiperparametr.

In [ ]:
grid_search = GridSearchCV(full_pipeline, hiperparam_grid, cv=5, scoring='accuracy')

No i teraz trochę sobie poczekamy:

In [16]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        Pipeline(steps=[('normalizer',
                                                         Normalizer())])),
                                       ('classification',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'classification__max_features': range(4, 14, 5),
                          'classification__n_estimators': (16, 32, 64),
                          'preprocessing__normalizer__norm': ('l1', 'l2',
                                                              'max')},
                         {'classification__max_features': ('sqrt', 'log2'),
                          'classification__n_estimators': (64, 128, 256),
                          'preprocessing__normalizer__norm': ('l1', 'l2',
                                                              'max')}],
             scoring='accuracy')

Aby oszacować czas wykonania `GridSearchCV` dla podanych hiperparametrów i zbioru **MNIST_784**, musimy zrozumieć, ile kombinacji hiperparametrów zostanie przetestowanych, jak długo trwa jedna iteracja treningu i jak wiele iteracji będzie wykonanych ze względu na walidację krzyżową.

### 1. **Ilość kombinacji hiperparametrów**
Zadeklarowany `hiperparam_grid` zawiera dwa zestawy parametrów:
```python
hiperparam_grid = [
    {
        'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
        'classification__n_estimators': (16, 32, 64),
        'classification__max_features': range(4, 14, 5),
    },
    {
        'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
        'classification__n_estimators': (64, 128, 256),
        'classification__max_features': ('sqrt', 'log2'),
    }
]
```

**Obliczenie liczby kombinacji w każdej siatce:**
1. **Pierwszy słownik:**
   - `preprocessing__normalizer__norm`: 3 wartości (`l1`, `l2`, `max`).
   - `classification__n_estimators`: 3 wartości (16, 32, 64).
   - `classification__max_features`: 3 wartości (4, 9, 14 z `range(4, 14, 5)`).
   - Liczba kombinacji: $( 3 \times 3 \times 3 = 27 )$.

2. **Drugi słownik:**
   - `preprocessing__normalizer__norm`: 3 wartości (`l1`, `l2`, `max`).
   - `classification__n_estimators`: 3 wartości (64, 128, 256).
   - `classification__max_features`: 2 wartości (`sqrt`, `log2`).
   - Liczba kombinacji: $( 3 \times 3 \times 2 = 18 )$.

**Łączna liczba kombinacji hiperparametrów:**
$
27 + 18 = 45
$

### 2. **Walidacja krzyżowa (cross-validation)**
- Ustawiono `cv=5`, co oznacza, że dla każdej kombinacji hiperparametrów model będzie trenowany 5 razy (na różnych podzbiorach danych).
- Łączna liczba treningów:
$
45 \times 5 = 225
$

### 3. **Czas trwania jednej iteracji treningu**
- **MNIST_784** ma 60,000 próbek treningowych i 10,000 testowych, a każdy obraz to wektor o wymiarach 784.
- Trening **RandomForestClassifier** na dużych zbiorach danych jest dość czasochłonny, zależnie od parametrów:
  - **Liczba drzew**: większa liczba drzew (`n_estimators`) zwiększa czas treningu.
  - **Max features**: wpływa na liczbę cech używanych w podziałach, co może zwiększyć lub zmniejszyć czas.
  
Dla przykładu:
- Trening lasu losowego z domyślnymi ustawieniami (100 drzew) na pełnym zbiorze MNIST może trwać około **20-40 sekund** na nowoczesnym procesorze.
- Ponieważ wartości `n_estimators` wynoszą od 16 do 256, czas treningu może wahać się od kilku do kilkudziesięciu sekund na jedną iterację.

Przyjmijmy średni czas jednej iteracji na **30 sekund**.

### 4. **Łączny czas obliczeń**
$
225 \, \text{iteracji} \times 30 \, \text{sekund} = 6750 \, \text{sekund} \approx 1,88 \, \text{godziny}
$

### 5. **Uwagi:**
- **Sprzęt**: Czas wykonania zależy od dostępnego sprzętu (CPU, RAM) i możliwości równoległego przetwarzania (`n_jobs` w `GridSearchCV`).
- **Optymalizacja**: Jeśli `n_jobs=-1` zostanie ustawione, procesy mogą być wykonywane równolegle, co znacznie skróci czas wykonania (np. na 4-rdzeniowym procesorze czas może spaść nawet do około 30-45 minut).

### Podsumowanie:
Na przeciętnym sprzęcie i bez równoległego przetwarzania, **GridSearchCV** dla podanego `hiperparam_grid` i zbioru **MNIST_784** może trwać około **1,5 do 2 godzin**. Z równoległym przetwarzaniem ten czas można znacznie skrócić.

U nas podziałało w około 1h 49 minut.

Za pomocą atrybutu *best_params_* można uzyskać najlepszą znalezioną kombinację hiperparametrów.

In [17]:
grid_search.best_params_

{'classification__max_features': 'sqrt',
 'classification__n_estimators': 256,
 'preprocessing__normalizer__norm': 'max'}

Obiekt klasy *GridSearchCV* może zostać z powodzeniem wykorzystany jako estymator podczas wyznaczania decyzji dla danych testowych.

In [18]:
y_pred = grid_search.predict(X_test)

Z łatwością można zauważyć, że deterministyczna metoda przeszukiwania siatki polega na sprawdzaniu wszystkich kombinacji. Dla trzech hiperparametrów, z których każdy ma zbiór trzech wartości do sprawdzenia, należy wykonać 9 cykli trening-test (zasada mnożenia). Przy 5-krotnej walidacji krzyżowej liczba cykli wzrasta do 135.

Do uzyskania bezpośredniego wyniku dla danych testowych można wykorzystać także metodę *score*.

In [19]:
grid_search.score(X_test, y_test)

0.9687142857142857

## Metoda losowego przeszukiwania siatki

W przypadku dużych przestrzeni hiperparametrów, czas ich optymalizacji może być bardzo długi. Metoda ta sprawdza się więc w przypadku niewielkich przestrzeni. W przypadku większych znacznie lepiej sprawdza się **metoda losowego przesukiwania siatki**, która polega na stochastycznym podpróbkowaniu przestrzeni hiperparametrów, co wpływa na zmniejszenie liczby cykli trening-test i - co się z tym wiąże - na skrócenie czasu trwania operacji. Do realizacji procesu losowego przeszukiwania siatki w bibliotece Scikit-learn przeznaczona jest klasa [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

In [20]:
import math
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

Warto mieć na uwadze fakt, że w przypadku korzystania z klasy *RandomizedSearchCV* należy wskazać zakres losowego próbkowania poszczegolnych hiperparametrów zamiast zbioru ich wartości.

In [21]:
hiperparam_distributions = {
    'preprocessing__normalizer__norm': ('l1', 'l2', 'max'),
    'classification__n_estimators': randint(16, 256),
    'classification__max_features': randint(4, int(math.sqrt(len(X[0]))))
}

In [22]:
randomized_search = RandomizedSearchCV(
    full_pipeline,
    hiperparam_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
)

W przypadku **`RandomizedSearchCV`**, czas wykonania zależy od liczby losowych prób (`n_iter`) określonych w parametrze. W kodzie ustawiono:

```python
n_iter=10
```

Oznacza to, że zamiast testować wszystkie 45 kombinacji hiperparametrów (jak w przypadku `GridSearchCV`), **`RandomizedSearchCV`** przetestuje tylko **10 losowych kombinacji**. To znacząco zmniejsza czas obliczeń.

### 1. **Liczba iteracji**
Liczba iteracji dla `RandomizedSearchCV` to po prostu:
$
\text{10} \times \text{cv} = 10 \times 5 = 50
$
Oznacza to, że model będzie trenowany 50 razy (w porównaniu do 225 razy w `GridSearchCV`).

### 2. **Czas trwania jednej iteracji**
Przyjmijmy, że średni czas treningu jednej iteracji wynosi **30 sekund** (jak wcześniej obliczono dla `RandomForestClassifier` na zbiorze **MNIST_784**).

### 3. **Łączny czas obliczeń**
$
50 \, \text{iteracji} \times 30 \, \text{sekund} = 1500 \, \text{sekund} = 25 \, \text{minut}
$

### 4. **Uwagi dotyczące sprzętu i optymalizacji**
- **Sprzęt**: Jeśli użyjesz parametru `n_jobs=-1`, procesy będą wykonywane równolegle, co dodatkowo skróci czas obliczeń.
- **Równoległość**: Na przykład na 4-rdzeniowym procesorze czas może wynosić około **6-8 minut** przy równoległym przetwarzaniu.

### Porównanie z `GridSearchCV`
- **`GridSearchCV`**: Ok. **1,5–2 godziny**.
- **`RandomizedSearchCV`**: Ok. **25 minut** bez równoległości, a z równoległością około **6–8 minut**.

### Dlaczego `RandomizedSearchCV` jest szybszy?
Zamiast testować wszystkie możliwe kombinacje hiperparametrów, `RandomizedSearchCV` losowo próbuje tylko niewielką podgrupę, co pozwala zaoszczędzić czas obliczeniowy. W praktyce często osiąga porównywalne wyniki przy znacznie mniejszym nakładzie obliczeń, zwłaszcza przy dużych przestrzeniach hiperparametrów.

In [23]:
randomized_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              Pipeline(steps=[('normalizer',
                                                               Normalizer())])),
                                             ('classification',
                                              RandomForestClassifier(random_state=42))]),
                   param_distributions={'classification__max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022E47B26D40>,
                                        'classification__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000022E47B276D0>,
                                        'preprocessing__normalizer__norm': ('l1',
                                                                            'l2',
                                                                            'max')},
                   random_state=42, scoring='accuracy')

U nas wykonywał się 52 minuty. Jest to 2x więcej niż przewidywanie, ale też2x krócej niż `GridSearchCv`.

### Dlaczego nasz czas różni się od wyliczonego?
W przypadku RandomizedSearchCV, wartości losowo wybrane w n_iter=10 mogły zawierać:
* Duże wartości n_estimators: Jeśli losowo wybrane kombinacje miały np. n_estimators=256 częściej niż w GridSearchCV, czas jednej iteracji mógł wzrosnąć.
* Więcej cech (max_features): Wyższe wartości cech (np. 28 w randint(4, 28)) zwiększają obciążenie przy dzieleniu węzłów drzewa, co wydłuża czas treningu.
W GridSearchCV czas wykonania jest bardziej przewidywalny, ponieważ wszystkie kombinacje są testowane, a liczba drzew (np. n_estimators=16, 32, 64) jest relatywnie niska w porównaniu do maksymalnej wartości losowanej w RandomizedSearchCV.

RandomizedSearchCV jest losowy, więc konkretne wartości hiperparametrów mogą być czasochłonniejsze w porównaniu do regularnych wartości z GridSearchCV. Przykłady:
* Jeśli większość losowo wybranych wartości to wyższe n_estimators i max_features, czas przetwarzania się wydłużył.
* Dodatkowe czynniki (np. większa liczba głębokich drzew w losowo wybranych konfiguracjach) mogły wydłużyć czas treningu.

Dostęp do najbardziej optymalnych hiperparametrów modelu oraz metody wyznaczania prognoz dla nowych obiektów pozostaje ten sam, co w przypadku klasy *GridSearchCV*.

In [24]:
randomized_search.best_params_

{'classification__max_features': 22,
 'classification__n_estimators': 230,
 'preprocessing__normalizer__norm': 'max'}

In [25]:
y_pred = randomized_search.predict(X_test)

In [26]:
randomized_search.score(X_test, y_test)

0.9691428571428572

## Zadania

1. Pobrać zbiór danych *Fashion-MNIST* (https://www.openml.org/search?type=data&status=active&id=40996) za pomocą funkcji *fetch_openml* (lub jeżeli będą problemy ściągnąć z pomocą kaggle - obojętnie czy plik .csv czy funkcją od kaggle: https://www.kaggle.com/datasets/zalando-research/fashionmnist/data), a następnie porównać wyniki dokładności, precyzji, pełności i czułości dla hiperparametrów lasu losowego i algorytmu k-NN za pomocą deterministycznej i stochastycznej metody przeszukiwania siatki.
2. Która z metod szybciej znalazła rozwiązanie?
3. Który z zestawów hiperparametrów zapewnił dokładniejsze wyniki klasyfikacji dla danych testowych?